In [3]:
!pip install psaw
!pip install jsonlines

--- Logging error ---
Traceback (most recent call last):
  File "C:\Users\Heet\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.7_qbz5n2kfra8p0\LocalCache\local-packages\Python37\site-packages\pip\_internal\utils\logging.py", line 177, in emit
    self.console.print(renderable, overflow="ignore", crop=False, style=style)
  File "C:\Users\Heet\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.7_qbz5n2kfra8p0\LocalCache\local-packages\Python37\site-packages\pip\_vendor\rich\console.py", line 1673, in print
    extend(render(renderable, render_options))
  File "C:\Users\Heet\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.7_qbz5n2kfra8p0\LocalCache\local-packages\Python37\site-packages\pip\_vendor\rich\console.py", line 1305, in render
    for render_output in iter_render:
  File "C:\Users\Heet\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.7_qbz5n2kfra8p0\LocalCache\local-packages\Python37\site-packages\pip\_internal\utils\logging.py", line 134, in __ric

--- Logging error ---
Traceback (most recent call last):
  File "C:\Users\Heet\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.7_qbz5n2kfra8p0\LocalCache\local-packages\Python37\site-packages\pip\_internal\utils\logging.py", line 177, in emit
    self.console.print(renderable, overflow="ignore", crop=False, style=style)
  File "C:\Users\Heet\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.7_qbz5n2kfra8p0\LocalCache\local-packages\Python37\site-packages\pip\_vendor\rich\console.py", line 1673, in print
    extend(render(renderable, render_options))
  File "C:\Users\Heet\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.7_qbz5n2kfra8p0\LocalCache\local-packages\Python37\site-packages\pip\_vendor\rich\console.py", line 1305, in render
    for render_output in iter_render:
  File "C:\Users\Heet\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.7_qbz5n2kfra8p0\LocalCache\local-packages\Python37\site-packages\pip\_internal\utils\logging.py", line 134, in __ric

In [4]:
import requests
import pandas as pd
import re
import datetime as dt

In [5]:
def get_data(url):
    a = requests.get(url)
    return a.json()

In [6]:
import os
DATA_ROOT = '../data/'

from psaw import PushshiftAPI
api = PushshiftAPI()

In [7]:
#date to fetch submissions after
start_epoch=int(dt.datetime(2018, 1, 1).timestamp())

In [8]:
subreddit_arr = ['ExplainLikeImFive', 'FoodForThought', 'ChangeMyView', 'TodayILearned']
submission_filter = ['id', 'subreddit','selftext',]
comment_filter = ['id', 'subreddit','body']
# id Id of the comment string
# subreddit Name of the subreddit string
# full_link Full link of the comment string
# body The body of the comment text_en
# author The original poster of the comment string
# parent_id Id of the comment/submission for which this
# comment is made.
# string
# parent_body The body of the parent comment/submission text_en Y
# is_submission A flag denoted is this record a submission or not boolean Y
# created_at Convert the created_utc to datetime format
# (YYYY-MM-DDThh:mm:ssZ)
# pdate Y

In [9]:
def getSubmission(items, subreddit_name = "Any", topic="None"):
    if subreddit_name == "Any" and topic != None:
        gen = api.search_submissions(
            filter=submission_filter, # list of fields to return
            after = start_epoch,
            q = topic,
            limit = items, # limit on the number of records returned
            num_comments=">10",
            selftextnot = "[removed]"
        )
    else:
        gen = api.search_submissions(
            subreddit=[subreddit_name], # one or more subreddits to include in the search
            filter=submission_filter, # list of fields to return
            limit = items, # limit on the number of records returned
            num_comments=">10",
            after = start_epoch,
            selftextnot = "[removed]"
        )
    return gen

In [10]:
def addFieldsSub(df,topic="None"):
# 1st fied: add topic
    topic_field = [topic]*len(df)
    df['topic'] = topic_field

# 2nd field: add is_sub
    isSub = [True]*len(df)
    df['full_link'] = df['url']
    df['isSubmission'] = isSub
    
# 3rd field: convert created_utc to human readable
    df['created_at'] = pd.to_datetime(df['created_utc'], utc=True, unit='s')
    df['created_at'] = df.created_at.dt.strftime('%d/%m/%y %H:%M:%S')
    
    df.drop('created_utc', axis=1, inplace=True)
    df.drop('created', axis=1, inplace=True)
    df.drop('num_comments', axis=1, inplace=True)
    df.drop('score', axis=1, inplace=True)
    df.drop('d_', axis=1, inplace=True)
    df.drop('url', axis=1, inplace=True)
    
    coms = []
    for index in range(len(df)):
        if (("[removed]" == df["selftext"].iloc[index]) or ("[deleted]" == df["selftext"].iloc[index])):
            coms.append(index)
    df.drop(labels=coms, axis=0, inplace=True)
    df_final = df.reset_index(drop=True)
    return df_final

In [11]:
def addFieldsCom(df, df1):
# 1st fied: add add parent body
    df = pd.DataFrame(df)
    df['parent_body'] = ["dummy"]*len(df)
    print(type(df))
    i = 0
    for index in range(len(df)):
        key = df['parent_id'].iloc[index][3:]
        df['parent_body'].iloc[index] = df1['selftext'].loc[df1['id']==key]
        #print(df['parent_body'].iloc[index])
        #df['parent_body'].iloc[index] = df['parent_body'].iloc[index].value.tolist()[0]
    
# 3rd field: convert created_utc to human readable
    df['created_at'] = pd.to_datetime(df['created_utc'], utc=True, unit='s')
    df['created_at'] = df.created_at.dt.strftime('%d/%m/%y %H:%M:%S')
    
# 4th field: Full link
    df['prefix'] = ["https//:reddit.com"]*len(df)
    df['full_link'] = df['prefix'] + df['permalink']
    
#  Drop fields
    df.drop('created_utc', axis=1, inplace=True)
    df.drop('created', axis=1, inplace=True)
    df.drop('score', axis=1, inplace=True)
    df.drop('d_', axis=1, inplace=True)
    df.drop('prefix', axis=1, inplace=True)
    df.drop('permalink', axis=1, inplace=True)
    df.drop('created_at',axis=1, inplace=True)
    df.drop('author',axis=1, inplace=True)    
    return df

In [68]:
political_submissions_df = pd.DataFrame(list(getSubmission(500, "Any","Joe Biden|Republican party|President")))
political_submissions_df = addFieldsSub(political_submissions_df, "Politics")
political_submissions_df  

C:\Users\Heet\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.7_qbz5n2kfra8p0\LocalCache\local-packages\Python37\site-packages\psaw\PushshiftAPI.py:252: UserWarning: Not all PushShift shards are active. Query results may be incomplete
  warnings.warn(shards_down_message)


,author,id,selftext,subreddit,title,topic,full_link,isSubmission,created_at
0,rulepanic,ywehto,,UkrainianConflict,POLISH PRESIDENT DUDA SAYS WHAT HAPPENED WAS A...,Politics,https://twitter.com/TommyLundn/status/15926656...,True,16/11/22 00:28:16
1,spartachilles,w9s1q6,"Ever since the Second American Revolution, few...",Presidentialpoll,Social Democratic Convention of 1920 | A House...,Politics,https://www.reddit.com/r/Presidentialpoll/comm...,True,27/07/22 22:28:27
2,He_DidNothingWrong,ubt9p2,,interestingasfuck,"In France, on Election Day, it's tradition tha...",Politics,https://v.redd.it/3fdvhii68qv81,True,25/04/22 19:29:38
3,The_Chubby_Unicorn,ubt3f8,According to the Fox News exclusive:\n\n“The f...,DWAC_Stock,Trump to start Truthing within the next seven ...,Politics,https://www.reddit.com/r/DWAC_Stock/comments/u...,True,25/04/22 19:22:09
4,dniprshyrokyi,ubsbn8,,UkrainianConflict,"Aleksey Arestovich, adviser to the head of the...",Politics,https://socportal.info/en/news/an-expert-who-p...,True,25/04/22 18:49:09
...,...,...,...,...,...,...,...,...,...
486,Kohenlevite,ti3l2w,"Hello, I just watched watching winter on fire:...",ukraine,"My changed views on Ukraine, and questions fro...",Politics,https://www.reddit.com/r/ukraine/comments/ti3l...,True,19/03/22 20:05:52
487,EnvironmentalAd6108,ti3kq2,My favourite villain each season\n\nMurder Hou...,AmericanHorrorStory,Best Villain each season,Politics,https://www.reddit.com/r/AmericanHorrorStory/c...,True,19/03/22 20:05:24
488,CriticalJossi-GPT2,ti3gs6,,SubSimGPT2Interactive,Niger's President has said he will resign beca...,Politics,https://static.reuters.com/resources/r/?m=02&a...,True,19/03/22 20:00:26
489,ZeldaLover2018,ti3fdz,"He caused a war between schools, was constantl...",FinalFantasyVIII,Why was Seifer still able to walk free after e...,Politics,https://www.reddit.com/r/FinalFantasyVIII/comm...,True,19/03/22 19:58:52


In [74]:
list_of_political_submissions = political_submissions_df['title'].to_list()
list_of_political_submissions.

['POLISH PRESIDENT DUDA SAYS WHAT HAPPENED WAS A ONE OFF INCIDENT -rtrs',
 'Social Democratic Convention of 1920 | A House Divided Alternate Elections',
 "In France, on Election Day, it's tradition that Journalists on Bikes try to chase the President-elect at high speed hurtling through Paris' streets until he gives his victory speech. Last night was no exception",
 'Trump to start Truthing within the next seven days',
 'Aleksey Arestovich, adviser to the head of the office of the President of Ukraine, expects the end of the active phase of the war with Russia in two weeks.',
 'Former Boston police union president pleads guilty to child rape charges',
 'Free speech',
 "Journalists following French President-elect Emmanuel Macron at high speed hurtling through Paris' streets",
 'Tony Khan really needs to appoint a AEW president and have him do all the major announcements.',
 "Joe Manchin's approval rating is up 16 points since Biden became president",
 'Discussion: When does the spirit 

In [69]:
comments_lst1 = []
for i in range(286):
    comments_lst1.extend(api.search_comments(link_id= political_submissions_df.iloc[i]['id'], subreddit= political_submissions_df.iloc[i]["subreddit"],
                                            filter=['id','parent_id','permalink','author', 'title', 
                                                    'subreddit','body','num_comments','score'],
                                            bodynot = "[removed]",
                                            limit=2     
                                            ))
comments1_df = pd.DataFrame(comments_lst1)
comments1_df

C:\Users\Heet\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.7_qbz5n2kfra8p0\LocalCache\local-packages\Python37\site-packages\psaw\PushshiftAPI.py:252: UserWarning: Not all PushShift shards are active. Query results may be incomplete
  warnings.warn(shards_down_message)
C:\Users\Heet\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.7_qbz5n2kfra8p0\LocalCache\local-packages\Python37\site-packages\psaw\PushshiftAPI.py:192: UserWarning: Got non 200 code 429
  warnings.warn("Got non 200 code %s" % response.status_code)
C:\Users\Heet\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.7_qbz5n2kfra8p0\LocalCache\local-packages\Python37\site-packages\psaw\PushshiftAPI.py:180: UserWarning: Unable to connect to pushshift.io. Retrying after backoff.
  warnings.warn("Unable to connect to pushshift.io. Retrying after backoff.")


,author,body,created_utc,id,parent_id,permalink,score,subreddit,created,d_
0,[deleted],[removed],1668624189,iwmec8j,t1_iwj5ydv,/r/UkrainianConflict/comments/ywehto/polish_pr...,1,UkrainianConflict,1.668642e+09,"{'author': '[deleted]', 'body': '[removed]', '..."
1,KrzysztofKietzman,And this is precisely why it shouldn't be maki...,1668607316,iwl9two,t1_iwky7px,/r/UkrainianConflict/comments/ywehto/polish_pr...,1,UkrainianConflict,1.668625e+09,"{'author': 'KrzysztofKietzman', 'body': 'And t..."
2,spartachilles,"Yes, without mass troop movements the spread w...",1658965021,ihx6z45,t1_ihx2zv2,/r/Presidentialpoll/comments/w9s1q6/social_dem...,1,Presidentialpoll,1.658983e+09,"{'author': 'spartachilles', 'body': 'Yes, with..."
3,Crisp_Leaves22,If the party right and center insist on Buchan...,1658963422,ihx3a2v,t3_w9s1q6,/r/Presidentialpoll/comments/w9s1q6/social_dem...,1,Presidentialpoll,1.658981e+09,"{'author': 'Crisp_Leaves22', 'body': 'If the p..."
4,Superdry_Wit,"Just like princess diana, what fun",1650998293,i6asylk,t3_ubt9p2,/r/interestingasfuck/comments/ubt9p2/in_france...,1,interestingasfuck,1.651016e+09,"{'author': 'Superdry_Wit', 'body': 'Just like ..."
...,...,...,...,...,...,...,...,...,...,...
565,[deleted],[removed],1647813982,i1ggtes,t3_tir94u,/r/ukraine/comments/tir94u/president_zelensky_...,1,ukraine,1.647832e+09,"{'author': '[deleted]', 'body': '[removed]', '..."
566,Dark-X,/r/IsraelExposed,1648137451,i1y0tw0,t1_i1m4s5m,/r/UkrainianConflict/comments/tir8mp/president...,1,UkrainianConflict,1.648155e+09,"{'author': 'Dark-X', 'body': '/r/IsraelExposed..."
567,extreminator,"Keep moving, troll",1648067589,i1ud7ir,t1_i1nfx6o,/r/UkrainianConflict/comments/tir8mp/president...,1,UkrainianConflict,1.648086e+09,"{'author': 'extreminator', 'body': 'Keep movin..."
568,RaytheonAcres,Real president heads know,1648048415,i1sye47,t3_tir7eu,/r/SubredditDrama/comments/tir7eu/are_these_pr...,1,SubredditDrama,1.648066e+09,"{'author': 'RaytheonAcres', 'body': 'Real pres..."


In [51]:
political_comments_lst = []
political_comments_lst.extend(api.search_comments(q=['Joe Biden|Republican party|President'],
                                        filter=['id','parent_id','permalink','author', 'title', 
                                                    'subreddit','body','num_comments','score'],
                                            bodynot = "[removed]",
                                            limit=10000))
political_comments_df = pd.DataFrame(political_comments_lst)
political_comments_df

C:\Users\Heet\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.7_qbz5n2kfra8p0\LocalCache\local-packages\Python37\site-packages\psaw\PushshiftAPI.py:252: UserWarning: Not all PushShift shards are active. Query results may be incomplete
  warnings.warn(shards_down_message)
C:\Users\Heet\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.7_qbz5n2kfra8p0\LocalCache\local-packages\Python37\site-packages\psaw\PushshiftAPI.py:192: UserWarning: Got non 200 code 429
  warnings.warn("Got non 200 code %s" % response.status_code)
C:\Users\Heet\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.7_qbz5n2kfra8p0\LocalCache\local-packages\Python37\site-packages\psaw\PushshiftAPI.py:180: UserWarning: Unable to connect to pushshift.io. Retrying after backoff.
  warnings.warn("Unable to connect to pushshift.io. Retrying after backoff.")


,author,body,created_utc,id,parent_id,permalink,score,subreddit,created,d_
0,Narrow_Muscle9572,"Does it do the same with the Royal family, cel...",1670268005,iz1d2ze,t3_zdg9gg,/r/lostredditors/comments/zdg9gg/how_is_that_o...,1,lostredditors,1.670286e+09,"{'author': 'Narrow_Muscle9572', 'body': 'Does ..."
1,TacosYBurritos2020,The Al Capone president,1670267998,iz1d2e9,t3_zd7mue,/r/politics/comments/zd7mue/trump_had_hidden_1...,1,politics,1.670286e+09,"{'author': 'TacosYBurritos2020', 'body': 'The ..."
2,Confident_Tension_75,The guy running for President.,1670267986,iz1d1av,t1_iz0qlc1,/r/mildlyinfuriating/comments/zd8342/kindergar...,1,mildlyinfuriating,1.670286e+09,"{'author': 'Confident_Tension_75', 'body': 'Th..."
3,GameboyPATH,They *are* screened. Thoroughly. By the FBI. A...,1670267980,iz1d0qy,t1_iz0otnq,/r/centrist/comments/zd8nfn/evidence_of_trump_...,1,centrist,1.670286e+09,"{'author': 'GameboyPATH', 'body': 'They *are* ..."
4,Ambientus,"Did you just wake up from a coma? Dude, I have...",1670267950,iz1cxup,t3_zdclyk,/r/TimDillon/comments/zdclyk/ben_says_he_was_f...,1,TimDillon,1.670286e+09,"{'author': 'Ambientus', 'body': 'Did you just ..."
...,...,...,...,...,...,...,...,...,...,...
9945,Maleficent-Photo6430,VP: Thompson\n\nVisits: mostly Washington but ...,1670164056,iyvifvy,t3_zcb0y5,/r/thecampaigntrail/comments/zcb0y5/buchanansl...,1,thecampaigntrail,1.670182e+09,"{'author': 'Maleficent-Photo6430', 'body': 'VP..."
9946,ayatoilet,"Look my friend, if you don't know anything abo...",1670164040,iyvieq1,t1_iyvhaph,/r/N_N_N/comments/zc2uhd/a_fragment_of_the_rus...,1,N_N_N,1.670182e+09,"{'author': 'ayatoilet', 'body': 'Look my frien..."
9947,xomns13,I'd probably leave the republican party after ...,1670164031,iyvie82,t1_iyva5hf,/r/PublicFreakout/comments/zc598d/this_is_patr...,1,PublicFreakout,1.670182e+09,"{'author': 'xomns13', 'body': 'I'd probably le..."
9948,Turbulent_Clerk4508,So the cost of Apple phones are up because of ...,1670164009,iyvicrc,t1_iyue50o,/r/worldwatch/comments/zbv9qt/biden_hopes_hes_...,1,worldwatch,1.670182e+09,"{'author': 'Turbulent_Clerk4508', 'body': 'So ..."


In [63]:
# import time
# # Get total replies to comments (includes replies to replies of comment)
# total_replies = {com_id: 0 for com_id in political_comments_df["id"].unique()}
# def increment_replies(row):
#         if row["link_id"] == row["parent_id"]: return
#         pid = row["parent_id"].split("_")[1]
#         if pid not in total_replies: return # Skip unrecorded comments
#         total_replies[pid] += 1
#         new_row = political_comments_df[political_comments_df["id"] == pid]
#         if len(new_row) > 1: print(len(new_row))
#         increment_replies(new_row.iloc[0])
#         for idx, row in political_comments_df.iterrows():
#             if   idx == 0: 
#                 time_start = time.time()
#                 if idx % 1000 == 0:
#                     print(str(idx) + " / " + str(len(political_comments_df)))
#                     increment_replies(row)

# political_comments_df["total_replies"] = political_comments_df["id"].apply(lambda x: total_replies[x])
# political_comments_df

,author,body,created_utc,id,parent_id,permalink,score,subreddit,created,d_,direct_replies,total_replies
0,Narrow_Muscle9572,"Does it do the same with the Royal family, cel...",1670268005,iz1d2ze,t3_zdg9gg,/r/lostredditors/comments/zdg9gg/how_is_that_o...,1,lostredditors,1.670286e+09,"{'author': 'Narrow_Muscle9572', 'body': 'Does ...",0,0
1,TacosYBurritos2020,The Al Capone president,1670267998,iz1d2e9,t3_zd7mue,/r/politics/comments/zd7mue/trump_had_hidden_1...,1,politics,1.670286e+09,"{'author': 'TacosYBurritos2020', 'body': 'The ...",0,0
2,Confident_Tension_75,The guy running for President.,1670267986,iz1d1av,t1_iz0qlc1,/r/mildlyinfuriating/comments/zd8342/kindergar...,1,mildlyinfuriating,1.670286e+09,"{'author': 'Confident_Tension_75', 'body': 'Th...",0,0
3,GameboyPATH,They *are* screened. Thoroughly. By the FBI. A...,1670267980,iz1d0qy,t1_iz0otnq,/r/centrist/comments/zd8nfn/evidence_of_trump_...,1,centrist,1.670286e+09,"{'author': 'GameboyPATH', 'body': 'They *are* ...",0,0
4,Ambientus,"Did you just wake up from a coma? Dude, I have...",1670267950,iz1cxup,t3_zdclyk,/r/TimDillon/comments/zdclyk/ben_says_he_was_f...,1,TimDillon,1.670286e+09,"{'author': 'Ambientus', 'body': 'Did you just ...",0,0
...,...,...,...,...,...,...,...,...,...,...,...,...
9945,Maleficent-Photo6430,VP: Thompson\n\nVisits: mostly Washington but ...,1670164056,iyvifvy,t3_zcb0y5,/r/thecampaigntrail/comments/zcb0y5/buchanansl...,1,thecampaigntrail,1.670182e+09,"{'author': 'Maleficent-Photo6430', 'body': 'VP...",0,0
9946,ayatoilet,"Look my friend, if you don't know anything abo...",1670164040,iyvieq1,t1_iyvhaph,/r/N_N_N/comments/zc2uhd/a_fragment_of_the_rus...,1,N_N_N,1.670182e+09,"{'author': 'ayatoilet', 'body': 'Look my frien...",0,0
9947,xomns13,I'd probably leave the republican party after ...,1670164031,iyvie82,t1_iyva5hf,/r/PublicFreakout/comments/zc598d/this_is_patr...,1,PublicFreakout,1.670182e+09,"{'author': 'xomns13', 'body': 'I'd probably le...",0,0
9948,Turbulent_Clerk4508,So the cost of Apple phones are up because of ...,1670164009,iyvicrc,t1_iyue50o,/r/worldwatch/comments/zbv9qt/biden_hopes_hes_...,1,worldwatch,1.670182e+09,"{'author': 'Turbulent_Clerk4508', 'body': 'So ...",0,0


In [78]:
pol_df= pd.DataFrame(political_comments_df['body'])
political_list = pol_df['body']
import json

xyz = json.loads(political_list.to_json(orient = "records"))
#del xyz['data']['index']

json_object= json.dumps(xyz)
#parsed = json.loads(xyz)
#json_object = json.dumps(parsed, indent=4) 

with open("political.json", "w") as outfile:
    outfile.write(json_object)


In [46]:
question = []
answers = []
for i in pol_df['body']:
   if(i%2==0):
      answers.append(i)
   else:
      question.append(i)
question
# questions = pd.DataFrame(pol_df[::2])
# answers = pd.DataFrame(pol_df[1::2])
# questions
# answers
# politic = pd.concat([questions,answers],axis=1)

TypeError: not all arguments converted during string formatting

In [ ]:
# data
list_of_messages = []
questions = []
answers = []

data_final = data.values()
# print(data_final)
for conversation in data_final:
    conversation = conversation['messages']
    for data_a in conversation:
        if(type(data_a)==list):
            list_of_messages.extend(data_a)
print(len(list_of_messages))
for i in range(0, len(list_of_messages)):
    if i%2==0:
        questions.append(list_of_messages[i]["text"])
    else:
        answers.append(list_of_messages[i]["text"])
# print(questions)
# print("-----")
# print(answers)

In [70]:
for val in political_comments_df["id"]:
    if political_comments_df["id"][val]==political_submissions_df["id"][val]:
        print[political_comments_df]["body"]

KeyError: 'iz1d2ze'

In [17]:
political_comment = api.search_comments(link_id= 'w9s1q6', subreddit=['Presidentialpoll'])
print(political_comment)

<generator object PushshiftAPIMinimal._search at 0x00000194CC9F0948>
